# December 10, 2025

https://adventofcode.com/2025/day/10

In [269]:
import re
import numpy as np
import pandas as pd
from scipy.optimize import LinearConstraint, Bounds, milp

In [37]:
line = f'''[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {{3,5,4,7}}'''
#indicators, buttons, jolts = re.fullmatch( r'\[(.*)]\s(.*)\s\{(.*)\}', line )
m = re.fullmatch( r'\[(.*)]\s(.*)\s\{(.*)\}', line )
indicators, buttons, jolts = m[1], m[2], m[3]

In [38]:
def parse_text( text ):
    puzz = list()
    for line in text:
        m = re.fullmatch( r'\[(.*)]\s(.*)\s\{(.*)\}', line )
        indicators, buttons, jolts = m[1], m[2], m[3]
        puzz.append( {
            # brackets are not in match, so convert . to False (off) and # to True (on)
            "indicators" : [x == "#" for x in indicators],
            # for each (x0,x1,...,xn) remove the parens, split the numbers, and convert from string to int
            "buttons" : [ [int(x) for x in btn[1:-1].split(",") ] for btn in buttons.split() ],
            # braces are not in match, so split on , and convert to int
            "jolts" : [int(x) for x in jolts.split(",")]
        })
    return puzz
    

In [39]:
test_text = f'''[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {{3,5,4,7}}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {{7,5,12,7,2}}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {{10,11,11,5,10,5}}'''

test = test_text.split("\n")
test = parse_text(test)


In [40]:
fn = "../data/2025/10.txt"
with open(fn, "r") as file:
    puzz_text = file.readlines()
puzz = [ line.strip() for line in puzz_text ]
puzz = parse_text(puzz)


# Part 1

In [41]:
def find_best_solution( goal, buttons, current=None, presses_so_far = 0, best = 9999 ):
    '''
    current = status of lights after previous presses
    buttons = remaining buttons to decide about
    goal = end status of lights for a valid sequence of button presses
    presses_so_far = presses from prior buttons
    best = best solution discovered so far
    '''

    if current is None:
        current = [False] * len(goal)

    # We can't beat current solution, end recusion
    if presses_so_far >= best:
        return None

    # We have chosen all our button actions, check solution
    if len(buttons) == 0:
        if current == goal:
            return presses_so_far
        else:
            return None

    
    for i, toggles in enumerate(buttons):
        # best if we don't press this button
        local_best1 = find_best_solution( goal, buttons[i+1:], current, presses_so_far, best )
        if local_best1 is not None and local_best1 < best:
            best = local_best1

        # best if we DO press this button
        new_status = [ not ind if idx in toggles else ind for idx, ind in enumerate(current) ]
        local_best2 = find_best_solution( goal, buttons[i+1:], new_status, presses_so_far + 1, best )
        if local_best2 is not None and local_best2 < best:
            best = local_best2

    return best
        

        



In [42]:
for i in range(len(test)):
    print( find_best_solution(test[i]["indicators"], test[i]["buttons"]) )

2
3
2


In [43]:
def part1( puzz ):
    tot = 0
    for machine in puzz:
        tot += find_best_solution( machine["indicators"], machine["buttons"] )

    return tot


In [44]:
part1( test )

7

In [45]:
part1( puzz )

401

# Part 2

### Solution 1 -- too slow

In [68]:
def find_best_solution_joltage( goal, buttons, current=None, presses_so_far = 0, best = 9999, check_for_indicators=False, verbose=False ):
    '''
    current = status of lights after previous presses
    buttons = remaining buttons to decide about
    goal = end status of lights for a valid sequence of button presses
    presses_so_far = presses from prior buttons
    best = best solution discovered so far
    check_for_indicators = should we check that all necessary indicators are wired by at least one button?
    '''
    if verbose :
        print("")
        print( buttons )
        print( current )
    # We can't beat current solution, end recusion
    if presses_so_far >= best:
        return None

    if current is None:
        current = [0] * len(goal)
    else:
        # check for win
        if goal == current:
            if verbose:
                print("solution found!")
            return presses_so_far

        # check for fail (joltage too high)
        for g,c in zip(goal, current):
            if c > g:
                if verbose:
                    print("<<<too much joltage>>>")
                return None
            
        # check for fail (no way to increase joltage for an indicator)
        if check_for_indicators:
            #print(goal)
            #print(current)
            all_indicators_available = set([ind for btn in buttons for ind in btn])
            for b, (g,c) in enumerate(zip(goal,current)):
                if b not in all_indicators_available and g != c:
                    if verbose:
                        print("<<<indicator not available>>>")
                    return None


    # otherwise, keep recursing...

    for i, toggles in enumerate(buttons):
        # If this is the last button for a particular indicator, then we know how many times to press it
        mandatory_presses = None

        if check_for_indicators or True:
            remaining_toggles = [ind for btn in buttons[i+1:] for ind in btn]
        
            for ind in toggles:
                if ind not in remaining_toggles:
                    if mandatory_presses is None:
                        # this indicator requires a specific number of presses (possibly 0 presses)
                        mandatory_presses = goal[ind] - current[ind]
                        if verbose:
                            print(f'''*indicator {ind} requires {mandatory_presses} presses*''')
                    elif mandatory_presses != goal[ind] - current[ind]:
                        # conflicting requirements, no solution available
                        if verbose:
                            print(f'''<<conflict for indicator {ind}>>''')
                        return None

                       
        # best result if we don't press this button again
        # if we have mandatory presses greater than 0, this path is unavailable
        if mandatory_presses is None or mandatory_presses == 0:
            local_best1 = find_best_solution_joltage( goal, buttons[i+1:], current, presses_so_far, best, check_for_indicators=True, verbose=verbose )
            if local_best1 is not None and local_best1 < best:
                best = local_best1

        # best if we DO press this button again (and possibly more other times!)
        # if we have mandatory press count equal to 0, this path is unavailable
        if mandatory_presses is None or mandatory_presses > 0:
            press_count = mandatory_presses or 1

            new_status = [ jolt+press_count if idx in toggles else jolt for idx, jolt in enumerate(current) ]
            if mandatory_presses is None:
                local_best2 = find_best_solution_joltage( goal, buttons[i:], new_status, presses_so_far + press_count, best, check_for_indicators=False, verbose=verbose )
            else:
                local_best2 = find_best_solution_joltage( goal, buttons[i+1:], new_status, presses_so_far + press_count, best, check_for_indicators=False, verbose=verbose )
            if local_best2 is not None and local_best2 < best:
                best = local_best2

        if mandatory_presses is not None:
            break

    if verbose:
        print("===end of call===")
        print(buttons)
        print(current)
        print("=================\n")






    return best

In [47]:
def part2( puzz ):
    tot = 0
    for i, machine in enumerate(puzz):
        best = find_best_solution_joltage( machine["jolts"], machine["buttons"] )
        print(f'''Machine {i} best = {best}''')
        tot += best
    return tot

In [67]:
part2( test )

solution found!
Machine 0 best = 10
solution found!
solution found!
solution found!
Machine 1 best = 12
solution found!
solution found!
solution found!
solution found!
solution found!
solution found!
Machine 2 best = 11


33

In [ ]:
machine = puzz[2]
find_best_solution_joltage( machine["jolts"], machine["buttons"], verbose=False )

### Solution 2 - too slow
This version tries to rearrange buttons more optimally to quit early more often

In [ ]:
class Machine:
    count = int(0)

    def __init__(self, specs ):
        self.id = Machine.count
        Machine.count += 1
    
        if type(specs) is Machine:
            # copy from existing Machine
            self.buttons = specs.buttons
            self.goal = specs.goal
            self.len = specs.len
            self.jolts = specs.jolts.copy()
            self.presses = 0
            self.best =None
            self.map = specs.map.copy()

        else:
            # create from buttons and jolts data
            self.buttons = specs["buttons"]
            self.goal = specs["jolts"]
            self.len = len(self.goal)
            self.jolts = [0]*self.len
            self.presses = 0
            self.best = None

            tmp = [ ([b for b, btn in enumerate(self.buttons) if ind in btn]) for ind in range(self.len) ]
            self.map = pd.DataFrame( {"indicator":[i for i in range(self.len)],
                                    "buttons": tmp,
                                    "nbtn": [len(t) for t in tmp] })
            self._sort_map()

    def print(self):
        print("Machine", self.id)
        print("Goal:", self.goal)
        print("Jolts:", self.jolts)
        print("Map")
        print(self.map)
            
    def _sort_map(self):
        self.map.sort_values( "nbtn", inplace=True )

    def __str__(self):
        return "Machine " + str(self.id)

    def is_broken(self):
        '''check if any joltages exceed goal'''
        for g,j in zip(self.goal, self.jolts):
            if j>g:
                return True
        return False
    
    def is_solved(self):
        '''check if joltage is correct'''
        return self.goal == self.jolts
        
    def press_button( self, b, times=1 ):
        '''press a button 0+ times and update state'''
        self.jolts = [ jolt+times if ind in self.buttons[b] else jolt for ind, jolt in enumerate(self.jolts) ]
        self.presses += times

    def remove_button( self, b ):
        '''remove a button so it can't be considered for more presses'''
        self.map["buttons"] = [ [x for x in btn_list if x != b] for btn_list in self.map["buttons"] ]
        self.map["nbtn"] = [len(btn_list) for btn_list in self.map["buttons"] ]
        self._sort_map()

    def solve( self, verbose=False ):
        if self.is_solved():
            if verbose:
                print(self, "is already solved")
            return 0
                 
        for r in range(self.map.shape[0]):
            if verbose:
                print(self, " iteration ", r)
            ind = self.map["indicator"].iloc[r]
            nbtn = len(self.map["buttons"].iloc[r])
            
            if self.is_broken():
                if verbose:
                    print(self, "is broken")
                # oops! no solution available
                return None
    
            if nbtn == 0:
                p = self.goal[ind] - self.jolts[ind]
                if p > 0:
                    if verbose:
                        print(self, "indicator", ind, "cannot be solved")
                    return None
                else:
                    if verbose:
                        print(self, "indicator", ind, "is solved")
                continue
            
            btn = self.map["buttons"].iloc[r][0]

            if nbtn == 1:
                # this implies a specific number of button presses is necessary.
                # carry that out, then remove the button from consideration
                p = self.goal[ind] - self.jolts[ind]
                if verbose:
                    print(self, "indicator", ind, "requires", p, "presses of button", btn)
                self.press_button( btn, times=p )
                self.remove_button( btn )
                if verbose:
                    self.print()

                if self.is_solved():
                    if verbose:
                        print(self, "solution found (", self.presses, " presses)")
                    return self.presses
            
            else:
                # otherwise, try all possibilities
                # note that recursion ensures that all other buttons/indicators are solved too
                for p in range( self.goal[ind] - self.jolts[ind] + 1 ):
                    if self.best is not None and self.presses + p >= self.best:
                        # early stop, no way to beat best
                        return self.best

                    if verbose:
                        print(self, "Try ", p, " presses of button ", btn, "...")
                    hypo = Machine(self)
                    hypo.press_button( btn, times=p )

                    # check: if we press this button and it breaks the machine, we shouldn't try even more presses!
                    if hypo.is_broken():
                        if verbose:
                            print(self, "breaks!")
                        return None
                    hypo.remove_button( btn )

                    if verbose:
                        hypo.print()
                    hypo_best = hypo.solve(verbose=verbose)
                    if verbose:
                        print("... back to", self)
                    if hypo_best is not None:
                        tot_presses = self.presses +  hypo_best
                        if verbose:
                            print("with solution of", tot_presses)
                        if self.best is None or tot_presses < self.best:
                            if verbose:
                                print("new best!")
                            self.best = tot_presses

                # recursion has solved all later rows as well
                return self.best

        # No solution found
        return None

                




In [220]:
def part2( puzz ):
    tot = 0
    for i, spec in enumerate(puzz):
        print("Puzzle", i)
        Machine.count = 0
        m = Machine(spec)
        out = m.solve()
        print("Solution =", out)
        tot += out
    return out



In [221]:
part2(test)

Puzzle 0
Solution = 10
Puzzle 1
Solution = 12
Puzzle 2
Solution = 11


11

In [ ]:
Machine.count=0
m = Machine(puzz[1])
m.print()
m.solve(verbose=True)

In [264]:
def solve_joltage( buttons, jolts ):
    # Set up linear constraints
    # Aij = 1 if indicator i is wired by button j

    # milp = mixed integer linear programming
    # minimizes c'x subject to lb <= A'x <= ub
    # and L <= x_i <= U
    # with each x_i optionally constrained to integer values

    N = len(jolts)
    A = np.array( [ [int(ind in btn_list) for btn_list in buttons] for ind in range(N) ] )
    jolts = np.array( jolts )
    constraints = LinearConstraint( A, jolts, jolts )
    c = np.array( [1]*len(buttons) ) # c'x = sum(x_i) = total button presses
    bounds = Bounds(0, np.inf)

    integrality = np.full_like( c, True )

    res = milp( c, integrality=integrality, bounds=bounds, constraints=constraints )
    
    return sum(res["x"])

In [266]:
def part2( puzz ):
    return sum( [solve_joltage(m["buttons"], m["jolts"]) for m in puzz] )

In [267]:
part2(test)

33.0

In [268]:
part2(puzz)

15017.0